#### Document transformers
- 문서 로드를 마쳤다면, 로드된 문서가 사용자의 어플리케이션에 맞게끔 변환되길 원할 것이다.
- 가장 간단한 예시는 긴 문서를 작은 청크 단위로 쪼개서, 사용자 모델 Context Window에 맞게 분할하는 것 일 것이다.
- LangChain은 분할, 결합, 필터링 등 다양한 빌트인 문서 변환기를 가지고 있다. 
- https://python.langchain.com/docs/modules/data_connection/document_transformers/


##### Text splitters
- "semantically related" 것들을 묶어서 chunk 하고 싶다
- 1. 텍스트를 의미론적 청킹으로 분할한다
- 2. 특정한 크기에 도달할 떄 까지 합친다(함수가 받아 들일 수 있을 단위 까지)
- 3. 

In [2]:
with open('./state_of_the_union.txt') as f:
    state_of_the_union = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain'

-----

In [ ]:
# !pip install python-dotenv
#!pip install python-certifi-win32

In [16]:
import os, time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from dotenv import load_dotenv

from pymilvus import connections
from pymilvus import Collection, CollectionSchema
from pymilvus import FieldSchema, DataType
from pymilvus import utility


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Milvus
from langchain.text_splitter import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer

import sys
sys.path.append('D:/203_GenAI_IBM/Manual')

In [24]:
## load embedding model
# model_name = 'sentence-transformers/all-MiniLM-L6-v2' # Try out with different model

# embeddings = HuggingFaceEmbeddings(model_name=model_name)

## text splitter parmas
chunk_size = 1000
chunk_overlap = 150
separator = "\n"

## create text splicter instance
r_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

filename = './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf'
loader = PyPDFLoader(filename)
pages = loader.load()
print('len(pages): ', len(pages))


docs = r_splitter.split_documents(pages)
#print(docs)

len(pages):  575


In [5]:
"""
도커 
"""
from pymilvus import connections
connections.connect(
  alias="default",
  host="127.0.0.1", # YOUR IP
  port="19530"      # YOUR PORT

)

## check if collection exist
COLLECTION_NAME = "test01"
has = utility.has_collection(COLLECTION_NAME)
print(f"Does collection hello_milvus exist in Milvus: {has}")

## create collection
fields = [
    FieldSchema(name = "id", dtype = DataType.INT64, description = "Ids", is_primary = True, auto_id = False),
    FieldSchema(name = "content", dtype = DataType.VARCHAR, description = "Content texts", max_length = 768*8),
    FieldSchema(name = "embedding", dtype = DataType.FLOAT_VECTOR, description = "Embedding vectors", dim = 384)
]
schema = CollectionSchema(fields = fields, description = "content collection")
collection = Collection(name = COLLECTION_NAME, schema = schema)

## create index 
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 1024}
}
collection.create_index(field_name = "embedding", index_params = index_params)

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19530. Timeout)>